In [ ]:
import pandas as pd
import numpy as np
import calendar
from timeit import default_timer as timer
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X=pd.read_csv('top30.csv')

In [ ]:
data=pd.read_excel('card transactions.xlsx')

In [ ]:
y=data[['Fraud','Date']]

In [ ]:
X.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
X=X.iloc[:,:24]

In [ ]:
X['date']=data['Date']

In [ ]:
X.head()

,ave_amount_card_zip_count_3,ave_amount_card_zip_count_1,ave_amount_card_zip_count_0,ave_amount_card_zip_count_7,ave_amount_card_zip_count_14,median_amount_card_zip_count_3,median_amount_card_zip_count_14,median_amount_card_zip_count_0,median_amount_card_zip_count_1,median_amount_card_merchnum_count_14,median_amount_card_zip_count_30,median_amount_card_zip_count_7,median_amount_card_merchnum_count_30,ave_amount_Cardnum_count_14,ave_amount_Cardnum_count_3,sum_amount_card_zip_count_30,ave_amount_Cardnum_count_0,ave_amount_Cardnum_count_30,ave_amount_Cardnum_count_1,median_amount_Cardnum_count_0,median_amount_Cardnum_count_1,act/median_amount_Merchnum_count_1,act/median_amount_Merchnum_count_0,median_amount_Cardnum_count_3,date
0,6.546966,5.992519,5.668170,7.582144,7.076372,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,34.956101,30.630963,25321.570000,16.797628,33.789793,24.989317,3.62,3.620,0.967914,0.986376,3.62,2010-01-01
1,106.355000,106.355000,106.355000,106.355000,94.590000,51.71,31.42,51.71,51.71,126.53,31.42,51.71,126.53,621.460883,969.576929,662.130000,1284.967703,538.009019,1127.611253,1920.00,1341.025,1.000000,1.000000,542.53,2010-01-01
2,285.500000,285.500000,285.500000,285.500000,264.874444,229.95,206.88,229.95,229.95,250.18,206.88,229.95,247.86,816.409299,850.361427,3180.180000,793.786176,780.180688,838.409647,303.68,399.000,1.121028,1.121028,350.93,2010-01-01
3,5.258034,5.158783,4.997035,5.286674,5.260876,3.80,3.80,3.80,3.80,3.80,3.80,3.80,3.80,7.251856,6.599404,861019.430001,5.444532,7.469275,6.799033,3.80,3.800,0.967914,0.986376,3.80,2010-01-01
4,6.546966,5.992519,5.668170,7.582144,7.076372,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,34.956101,30.630963,25321.570000,16.797628,33.789793,24.989317,3.62,3.620,0.967914,0.986376,3.62,2010-01-01


In [ ]:
oot_x=X[X['date']>='2010-10-01']
oot_y=y[y['Date']>='2010-10-01']

In [ ]:
X=X[(X['date']>'2010-01-14') & (X['date']<'2010-10-01')]

In [ ]:
y=y[(y['Date']>'2010-01-14') & (y['Date']<'2010-10-01')]

In [ ]:
y=y.drop('Date',axis=1)
X=X.drop('date',axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30,random_state = 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)

## Boosted Tree

In [ ]:
import xgboost as xgb
xg_class = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 500)
xg_class.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='reg:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
training=pd.DataFrame()
y_pred=xg_class.predict_proba(X_train)
test_score=y_pred[:,1]
training['Fraud']=y_train['Fraud']
training['score']=test_score
numbads=len(y_train[y_train['Fraud']==1])
numbads

547

In [ ]:
topRows = int(round(len(training)*0.03))

temp = training[['score','Fraud']].copy()
temp0 = temp.sort_values('score',ascending=False)
temp1 = temp0.head(topRows)
temp2 = temp0.tail(topRows)
needed1 = temp1.loc[:,'Fraud']
needed2 = temp2.loc[:,'Fraud']
FDR1 = sum(needed1)/numbads
FDR2 = sum(needed2)/numbads
FDRate = np.maximum(FDR1,FDR2)
FDRate

0.9853747714808044

In [ ]:
X_test = mms.fit_transform(X_test)

In [ ]:
y_pred=xg_class.predict_proba(X_test)

In [ ]:
testing=pd.DataFrame()
test_score=y_pred[:,1]
testing['Fraud']=y_test['Fraud']
testing['score']=test_score
numbads=len(y_test[y_test['Fraud']==1])
numbads

231

In [ ]:
topRows = int(round(len(testing)*0.03))

temp = testing[['score','Fraud']].copy()
temp0 = temp.sort_values('score',ascending=False)
temp1 = temp0.head(topRows)
temp2 = temp0.tail(topRows)
needed1 = temp1.loc[:,'Fraud']
needed2 = temp2.loc[:,'Fraud']
FDR1 = sum(needed1)/numbads
FDR2 = sum(needed2)/numbads
FDRate = np.maximum(FDR1,FDR2)
FDRate

0.7012987012987013

In [ ]:
oot_x = oot_x.drop('date',axis=1)
oot_y = oot_y.drop('Date',axis=1)
oot_x = mms.fit_transform(oot_x)

In [ ]:
y_pred=xg_class.predict_proba(oot_x)
oot=pd.DataFrame()
test_score=y_pred[:,1]
oot['Fraud']=oot_y['Fraud']
oot['score']=test_score
numbads=len(y_test[y_test['Fraud']==1])
numbads

231

In [ ]:
topRows = int(round(len(testing)*0.03))

temp = oot[['score','Fraud']].copy()
temp0 = temp.sort_values('score',ascending=False)
temp1 = temp0.head(topRows)
temp2 = temp0.tail(topRows)
needed1 = temp1.loc[:,'Fraud']
needed2 = temp2.loc[:,'Fraud']
FDR1 = sum(needed1)/numbads
FDR2 = sum(needed2)/numbads
FDRate = np.maximum(FDR1,FDR2)
FDRate

0.43722943722943725